# Angelina Dubrule

## Research question/interests

**Does the population of the city that the property is located in have any affect on the affordablilty of a mortgage? If so, is this affect positive or negative?**

We can answer this question by averaging the affordability over the 17 year time span (1999-2016) for each row in the graph. Then we could compare this to the population ranking by removing all other columns. We can then combine the repeated data from the same city multiple times by averaging the repeated terms. Once there are only two columns, average affordability and rank, we can order the data by population rank and then plot the data in a graph to see if there are any trends.

**Which city in our dataset has the most affordable mortgage? Which city has the least?**

We can answer this question by averaging the affordability over the 17 year time span (1999-2016) for each row in the graph. Then we could compare this to each RegionName by removing all other columns. We can then combine the repeated data from the same city multiple times by averaging the repeated terms. Once there are only two columns, average affordability and Region Name, we can order the terms by lowest to hishest affordability. The first Region Name term will be the most affordable, and the last will be the least.

**Has the affordability of a mortgage changed over the 17 year period that the data was collected? If so, did affordability increase or decrease?**

We can answer this question by removing all data columns from the table except for the affordability calculation columns. Find the average for each quarter by finding the average for each column using all rows. There should now be only two rows, the date of data collecton and the average affordability. We can then plot a graph showing the average affordability over time, and see if there are any trends.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../data/raw/TieredAffordability_Mortgage.csv")